# Noisy 1B5 (Finetune Training)
We will be finetuning a 1B5 model, and intentionally overtrain it.
This is to measure how the introduction of a noisy state will impact train.

The goal of this experiment, is to measure two seperate objective
- test for overfitting conditions across multiple epoch
- test if overfitting is mitigated with the introduction of a noisy state

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

In [1]:
# First lets get the RWKV base model which we will be finetuning
!mkdir -p ../../../model/
!mkdir -p ../../../datapath/
!mkdir -p ../../../checkpoint/
!cd ../../../model/ && wget -nc https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth

File ‘Echo-A-1B5-Init.pth’ already there; not retrieving.



In [5]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="MultiEpoch-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_2_offload
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/picocreator-noise-experiment/notebook/experiment/noisy-training
TRAINER_DIR: /home/picocreator/rwkv-proj/picocreator-noise-experiment/RWKV-v4neo
PROJECT_DIR: /home/picocreator/rwkv-proj/picocreator-noise-experiment


In [ ]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/MultiEpoch-1B5-shakespeare-baseline.yaml"

## Stage 1 : Baseline model training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B5-shakespeare-baseline.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Shakespeare Baseline (ctx=2048, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B5-Shakespeare-Baseline/last.ckpt" "../model/MultiEpoch-1B5-Shakespeare-Baseline.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B5-Shakespeare-Baseline.pth"

## Stage 2 : Noisy prefix model training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B5-shakespeare-noisy.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Shakespeare Noisy (ctx=2048, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B5-Shakespeare-Noisy/last.ckpt" "../model/MultiEpoch-1B5-Shakespeare-Noisy.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B5-Shakespeare-Noisy.pth"

## Stage 3 : Noisy init state model training

In [6]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B5-shakespeare-rinit.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Shakespeare Random Init State (ctx=2048, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

[2023-08-06 17:33:19,810] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 976406050
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 976406050
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in ./wandb/run-20230806_173322-n68ifccg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run MultiEpoch-1B5 - Shakespeare Random Init State (ctx=2048, deepspeed_stage_2_offload)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Noise-Training
wandb: 🚀 View run at htt

In [7]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B5-Shakespeare-RInit/last.ckpt" "../model/MultiEpoch-1B5-Shakespeare-RInit.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B5-Shakespeare-RInit.pth"

[2023-08-06 17:57:41,956] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/MultiEpoch-1B5-Shakespeare-RInit/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.gradients, world_size: 1
Parsing checkpoint created by deepspeed==0.9.5
Reconstructed fp32 state dict with 438 params 1515106304 elements
Saving fp32 state dict to ../model/MultiEpoch-1B5-Shakespeare-RInit.pth
-rw-rw-r-- 1 picocreator picocreator 5.7G Aug  6 17:57 ../model/MultiEpoch-1B5-Shakespeare-RInit.pth
